<a href="https://colab.research.google.com/github/iliavrtn/blue-book-for-bulldozers-kaggle/blob/main/Llama_Cloud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PIP Installs

In [3]:
!pip install firebase

# Importing Modules

In [4]:
import requests
from bs4 import BeautifulSoup
import json
from firebase import firebase
from IPython.display import HTML, IFrame, display, Javascript, clear_output
from google.colab import output


# Config

In [5]:
database = "https://llama-index-bebra-default-rtdb.europe-west1.firebasedatabase.app/"
# Define the base URL and search words
base_url = 'https://aws.amazon.com/'
#search_words = ['AWS', 'Cloud Computing', 'EC2', 'S3', 'Lambda', 'IAM', 'VPC', 'RDS', 'Route', 'CloudFormation']
search_words = [
    'AWS', 'Cloud Computing', 'EC2', 'S3', 'Lambda', 'IAM', 'VPC', 'RDS', 'Route 53', 'CloudFormation',
    'DynamoDB', 'Elastic Beanstalk', 'S3 Glacier', 'EBS', 'EFS', 'Aurora', 'CloudWatch', 'CloudTrail',
    'Elastic Load Balancing', 'Auto Scaling', 'AWS CLI', 'SDK', 'API Gateway', 'Kinesis', 'Elasticsearch Service',
    'QuickSight', 'Redshift', 'SES', 'SNS', 'SQS', 'Fargate', 'Step Functions', 'WAF', 'Shield', 'Certificate Manager',
    'Direct Connect', 'AppSync', 'Managed Blockchain', 'Lightsail', 'RoboMaker', 'Timestream', 'Transcribe',
    'Translate', 'Polly', 'Rekognition', 'Lex', 'SageMaker', 'Security Hub', 'Secrets Manager', 'Systems Manager'
]
max_urls = 100
FBconn = firebase.FirebaseApplication(database, None)

# Scrapper Code

Function to run web scrapper and save results in DB and JSON

In [6]:
def scrape_and_save():
    # Initialize a dictionary to hold the results
    results = {}
    results['term'] = {word: [] for word in search_words}

    # Keep track of URLs to visit, those already visited, and the link count for ranking
    urls_to_visit = [base_url]
    visited_urls = set()
    url_link_count = {}  # For page ranking

    while urls_to_visit:
        url = urls_to_visit.pop(0)
        visited_urls.add(url)

        # Send a GET request to the website
        response = requests.get(url)

        if response.status_code == 200:
            # Use BeautifulSoup to parse the HTML content
            soup = BeautifulSoup(response.text, 'html.parser')

            # Extract title and a short content description
            title = soup.title.string if soup.title else 'No title found'
            content = soup.find('meta', {'name': 'description'})
            content = content['content'] if content else 'No description available'
            if not content:  # If meta description is empty, grab the first paragraph
                first_paragraph = soup.find('p')
                content = first_paragraph.text if first_paragraph else 'No content available'

            # Find all links in the webpage
            for link in soup.find_all('a', href=True):
                href = link['href']
                link_text = link.text.strip()
                full_url = requests.compat.urljoin(url, href)

                # Update page rank (link count)
                if full_url in url_link_count:
                    url_link_count[full_url] += 1
                else:
                    url_link_count[full_url] = 1

                # Check if the URL belongs to the base domain or its subdomains and hasn't been visited yet
                if 'aws.amazon.com' in full_url and full_url not in visited_urls:
                    if any(word.lower() in full_url.lower() for word in search_words):
                        urls_to_visit.append(full_url)

                    # Check if the link text or href contains any of the search words
                    for word in search_words:
                        if word.lower() in link_text.lower() or word.lower() in href.lower():
                            # Add the link to the results under the relevant search word
                            if full_url not in results['term'][word]:
                                results['term'][word].append({
                                    'DocIDs': full_url,
                                    'rank': url_link_count[full_url],
                                    'title': title,
                                    'content': content
                                })

        # Limit to prevent excessive recursion or too many requests; adjust as needed
        if len(visited_urls) >= max_urls:
            break

    # Sort results by page rank before saving
    for word in results['term']:
        results['term'][word].sort(key=lambda x: x['rank'], reverse=True)

    # Save the results to a JSON file
    with open('aws_links_ranked.json', 'w') as json_file:
        json.dump(results, json_file, indent=4)

    print("Scraping completed and results saved to aws_links_ranked.json.")

    # Saving to DB
    #FBconn = firebase.FirebaseApplication(database, None)  # Ensure you replace 'database' with your actual Firebase database URL.
    result = FBconn.post('/index/', results)
    print(result)


In [7]:
#Wipes the entire Index clean
def wipe_index():
  path_to_delete = '/index'
  # Delete all data at the specified path
  result = FBconn.delete(path_to_delete, '')

# Landing Page HTML

In [32]:
landing_page_html = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Llama</title>

    <!-- For PNG or other formats -->
    <link rel="icon" type="image/png" href="logo_nobg.png">
    <!-- Apple Touch Icon for use with Apple devices -->
    <link rel="apple-touch-icon" href="logo_nobg.png">
    <!-- To specify a theme color for address bar in mobile browsers -->
    <meta name="theme-color" content="#10182F">

    <!-- Function to change the title of the page when the tab is not active -->
    <script>
        function handleVisibilityChange() {
            if (document.hidden) {
                // Set the new title when the tab is inactive
                document.title = "Llama 😢";
            } else {
                // Reset the title when the tab is active again
                document.title = "Llama";
            }
        }
        // Add a listener for the visibilitychange event
        document.addEventListener("visibilitychange", handleVisibilityChange, false);
    </script>

    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }

        body,
        html {

            -webkit-user-select: none;
            /* Safari */
            -moz-user-select: none;
            /* Firefox */
            -ms-user-select: none;
            /* Internet Explorer/Edge */
            user-select: none;
            /* Non-prefixed version, currently supported by Chrome, Opera and Edge */

            height: 100%;
            font-family: 'Roboto', sans-serif;
            background: linear-gradient(to bottom left, #0F172A, #1A2346, #193251, #142941, #0F172A, #0F172A, #0F172A);
            /* Dark background */
            background-attachment: fixed;
            /* Keep the gradient fixed during scroll */
            color: white;
        }

        /* --- HEADER --- */
        header {
            position: absolute;
            display: flex;
            top: 0;
            width: 100%;
            padding: 1rem;
            justify-content: space-between;
            align-items: center;
        }

        header .login,
        header .user-icon {
            color: #94A3B8;
        }

        header .user-icon {
            font-size: 2rem;
        }

        .logo-container {
            top: 0;
            right: 0;
            padding: 10px;
            /* Adjust the padding to control the distance from the corner */
            display: flex;
            /* Enables Flexbox for this container */
            align-items: center;
            /* Vertically centers the flex items (the image and label) */
            justify-content: flex-start;
            /* Aligns items to the start of the container */
        }

        .logo {
            height: 100px;
            /* Adjust based on your logo's size */
            width: auto;
            /* Maintains the aspect ratio of the logo */
            margin-right: 10px;
            /* Adds some space between the logo and the label */
        }

        .miniTitle {
            font-size: 1.5rem;
            font-weight: 600;
            color: #38BDF8;
        }

        .title {
            font-size: 6rem;
            font-weight: 600;
            background: linear-gradient(to right, #007bff, #70bee6);
            /* Gradient from blue to light blue */
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            display: inline;
            /* Required for gradient text */
            background-clip: text;
            color: transparent;
            /* Fallback for non-webkit browsers */
            text-shadow: 0 0 3px #007bff59, 0 0 6px #007bff56, 0 0 9px #85d6ff50;
            /* Glowing effect */
            font-weight: bold;
        }

        .login-container {
            text-decoration: none;
            /* Remove underline from links */
            display: flex;
            gap: 6px;
            align-items: center;
            justify-content: center;
            color: #94A3B8;
            margin: 20px;
            background-color: #ffffff0f;
            border: 1px solid #ffffff1a;
            width: auto;
            min-width: 120px;
            border-radius: 10px;
            padding: 10px;
            backdrop-filter: blur(10px);
            position: fixed;
            right: 0;
            transition: background-color 0.3s ease;
            cursor: pointer;
            /* Make the cursor indicate clickable area */
        }

        .login-container:hover {
            background-color: #ffffff1a;
        }


        /* --- MAIN --- */

        main {
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            /* Use at least 100% of the viewport height */
            width: 100vw;
            /* Use 100% of the viewport width */
        }

        main .container {
            display: flex;
            flex-direction: column;
            justify-content: center;
            align-items: center;
            text-align: center;
            /* Change height to 100vh to fill the entire viewport */
        }

        .welcome-message {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            height: 30vh;
            /* This takes up the full height of the viewport */
            text-align: center;
            margin-bottom: 30px;
        }


        .welcome-message p {
            font-size: 1.25rem;
            /* Larger font size for the subtitles */
            margin-bottom: 1rem;
            /* Spacing between paragraphs */
            margin-top: 1rem;
        }

        .welcome-message .cta-text {
            font-size: 1.5rem;
            /* Even larger font size for call-to-action */
            font-weight: bold;
            /* Make it bold */
            color: #ff9900;
            /* Make it a color that stands out */
            animation: pulse 2s infinite;
            /* Add an animation to attract attention */
        }

        /* Keyframes for the pulse animation */
        @keyframes pulse {
            0% {
                transform: scale(1);
            }

            50% {
                transform: scale(1.05);
            }

            100% {
                transform: scale(1);
            }
        }


        .search-box {
            display: flex;
            /* This will align the children (input and button) in a row */
            align-items: center;
            /* This will vertically center the input and button */
            overflow: hidden;
            /* Clip the children to the wrapper's border radius */
            border-radius: 50px;
            /* Your desired border radius */

            border: 2px solid #0488D2;
            /* New: Unified border style */
            background: #ffffff1a;
            /* New: Unified background */
            /* box-shadow: 0 2px 4px #007bff88; */
            /* New: Default shadow, consider conditional application */

        }

        .search-box:focus-within {
            box-shadow: 0 0 8px #007bff;
            /* New box-shadow when input is focused */
            border-color: #007bff;
            /* Change border color on hover */
        }

        /* Small x in the search bar right corner */
        .search-box input[type="search"]::-webkit-search-cancel-button {
            -webkit-appearance: none;
            height: 0;
            width: 0;
            display: none;
        }

        .search-box:hover {
            border-color: #007bff;
            /* Change border color on hover */
            box-shadow: 0 0 8px #007bff88;
            /* Subtle shadow for depth */
        }

        .search-box input[type="search"] {
            color: #E2E8F0;
            padding: 1rem 0.5rem;
            font-size: 1rem;
            width: 30rem;
            font-weight: bold;
            border: none;
            /* Adjusted: Removed border */
            background: transparent;
            /* Adjusted: Background transparent */
        }

        .search-box input[type="search"]::placeholder {
            color: #94A3B8;
            font-weight: bold;
        }

        /* When pressed on search */
        .search-box input[type="search"]:focus {
            outline: none;
        }

        .search-box .search-btn,
        .left-pan {
            padding: 13px 20px;
            font-size: 1.3em;
            border: none;
            /* Adjusted: Removed individual borders */
            background: transparent;
            /* Adjusted: Background transparent */
            color: #94A3B8;
            cursor: pointer;
        }

        .search-box .search-btn i,
        .left-pan .divider {
            pointer-events: none;
            /* Prevents the icon from being clickable separately from the button */
        }

        .search-box .search-btn:hover {
            background: linear-gradient(to bottom left, #0F172A, #10182F);
            /* border-color: #0d6efd; */
            color: #E2E8F0;
        }

        .left-pan .divider {
            color: #94A3B8;
            /* Light grey color for the divider */
            font-size: 1.4rem;
            font-weight: bold;
        }

        .left-pan:hover {
            color: #E2E8F0;
            /* Change background color on hover */
            cursor: pointer;
        }

        .left-pan i.fa-microphone {
            /* Apply glow effect to the microphone icon */
            text-shadow: 0 0 2px #007bff;
        }

        .search-btn i.fa-search {
            /* Apply glow effect to the search icon */
            text-shadow: 0 0 3px #0F172A;
        }

        /* Optional: Enhance the glow on hover */
        .left-pan:hover i.fa-microphone {
            text-shadow: 0 0 12px #007bff;
            /* Stronger glow effect on hover */
        }


        .filter-options {
            display: flex;
            justify-content: space-evenly;
            align-items: center;
            /* This will vertically center the items in the container */
            gap: 2rem;
            /* Space between filter buttons */
            margin: 1.3rem 0;
            /* Distance from the search box */
            flex-wrap: wrap;
            /* Ensure responsiveness */
        }

        .filter-topic,
        .filter-btn {
            padding: 10px 15px;
            border: 2px solid #0488D2;
            /* Match the search box border */
            background-color: #ffffff1a;
            /* Semi-transparent background like the search box */
            color: #94A3B8;
            /* Light grey color to match the icon and placeholder */
            font-weight: bold;
            /* For emphasis */
            border-radius: 50px;
            /* Rounded corners */
            cursor: pointer;
            transition: background-color 0.3s, color 0.3s;
            /* Smooth transition for hover effects */
        }

        .filter-btn {
            background: linear-gradient(to bottom left, #0F172A, #1A2346);
            /* Gradient background for filter button */
            color: #E2E8F0;
            /* White-ish color for text to stand out */
        }

        .filter-topic:hover,
        .filter-btn:hover {
            background-color: #0488D2;
            /* Darker color on hover for visibility */
            color: #FFFFFF;
            /* White color for text on hover */
            border-color: #007bff;
            /* Slightly lighter border on hover */
        }

        .filter-topic.pressed {
            background-color: #0056b3;
            /* Darker shade to indicate pressed state */
            color: #FFFFFF;
            /* Change text color for contrast */
            border-color: #004080;
            /* Optionally change the border color */
            box-shadow: 0 0 4px #007bff88;
            /* Subtle shadow for depth */
        }


        .more-filters {
            display: flex;
            flex-direction: column;
            align-items: center;
            margin-top: 20px;
            padding: 10px;
            background-color: #0F172A;
            /* Deep background to contrast with button */
            border-radius: 10px;
            /* Rounded corners for the container */
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            /* Soft shadow for depth */
        }

        .checkbox-container {
            margin: 10px 0;
        }

        .checkbox-container input[type="checkbox"] {
            /* Hide the default checkbox */
            -webkit-appearance: none;
            -moz-appearance: none;
            appearance: none;
            /* Create a custom checkbox appearance */
            width: 20px;
            height: 20px;
            background-color: #ffffff1a;
            border: 2px solid #0488D2;
            border-radius: 4px;
            cursor: pointer;
            position: relative;
        }

        .checkbox-container input[type="checkbox"]:checked {
            background-color: #007bff;
            border: 2px solid #007bff;
        }

        .checkbox-container input[type="checkbox"]:checked::after {
            /* Custom checkmark style */
            content: '';
            position: absolute;
            left: 5px;
            top: 1px;
            width: 6px;
            height: 12px;
            border: solid white;
            border-width: 0 3px 3px 0;
            transform: rotate(45deg);
        }

        .checkbox-container label {
            margin-left: 8px;
            color: #94A3B8;
            /* Light grey color to match the design */
            font-weight: bold;
            user-select: none;
            /* Prevent text selection */
        }

        /* --- FOOTER --- */

        footer {
            position: absolute;
            bottom: 0;
            width: 100%;
            padding: 1rem;
            text-align: center;
            background: transparent;
        }

        footer a {
            color: #E2E8F0;
            /* Light grey color for footer links */
            text-decoration: none;
            font-size: 1rem;
            margin: 0 1rem;
            transition: color 0.3s ease;
        }

        footer a:hover {
            color: #0E99DA;
        }

        /* --- FOR DIFFERENT SIZES OF SCREEN --- */
        @media (max-width: 768px) {
            .title {
                font-size: 4rem;
                /* Smaller font size for smaller screens */
            }

            .miniTitle,
            .welcome-message p,
            .welcome-message .cta-text {
                font-size: 1rem;
                /* Adjust font size for readability */
            }

            .search-box input[type="search"] {
                font-size: 0.8rem;
                /* Smaller font size for the search input */
                width: 15rem;
                /* Adjust width to fit smaller screens */
            }

            header,
            .login-container,
            .welcome-message,
            main .container {
                padding: 0.5rem;
                /* Reduce padding */
                margin: 0.5rem;
                /* Adjust margins */
            }

            .logo {
                height: auto;
                /* Maintain aspect ratio */
                max-width: 15%;
                /* Ensure logo does not exceed container width */
            }

            .filter-options,
            .more-filters {
                display: none;
                /* Optionally hide filters on smaller screens */
            }
        }

        /* The Modal (background) */
        .modal {
            display: none;
            /* Hidden by default */
            position: fixed;
            /* Stay in place */
            z-index: 1;
            /* Sit on top */
            left: 0;
            top: 0;
            width: 100%;
            /* Full width */
            height: 100%;
            /* Full height */
            overflow: auto;
            /* Enable scroll if needed */
            background-color: rgba(0, 0, 0, 0.4);
            /* Black w/ opacity */
        }

        /* Modal Content/Box - Adjusted to match your system's CSS */
        .modal-content {
            background-color: #142941;
            /* Deep background */
            margin: 10% auto;
            /* Adjusted for centering */
            padding: 20px;
            border: 2px solid #0488D2;
            /* Border color to match your system */
            width: 40%;
            /* Adjust based on preference */
            border-radius: 10px;
            /* Rounded corners */
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.2);
            /* Soft shadow for depth */
            color: #E2E8F0;
            /* Text color */
        }

        /* The Close Button - Styled to fit within your design */
        .close {
            color: #aaa;
            float: right;
            font-size: 28px;
            font-weight: bold;
        }

        .close:hover,
        .close:focus {
            color: #E2E8F0;
            text-decoration: none;
            cursor: pointer;
        }

        /* Additional styles for inputs and buttons inside the modal to match your theme */
        .modal-content input[type="password"],
        .modal-content button {
            font-family: 'Roboto', sans-serif;
            padding: 10px 15px;
            margin-top: 10px;
            /* Space between elements */
            border: 2px solid #0488D2;
            /* Matching the search box border */
            background-color: #ffffff1a;
            /* Semi-transparent background */
            color: #94A3B8;
            /* Light grey color for text */
            font-weight: bold;
            border-radius: 50px;
            /* Rounded corners */
            cursor: pointer;
            width: calc(100% - 40px);
            /* Full width minus padding */
            box-sizing: border-box;
            /* Border box */
        }

        .modal-content button {
            background: linear-gradient(to bottom left, #0F172A, #1A2346);
            /* Gradient background */
            color: #E2E8F0;
            /* White-ish color for text */
        }

        /* Focus state for the password input */
        .modal-content input[type="password"]:focus {
            outline: none;
            border: 2px solid #E2E8F0;
            /* Highlight color */
        }

        /* Stylish hover animation for the button */
        .modal-content button:hover {
            background-color: #0488D2;
            /* Darker shade on hover */
            color: #FFFFFF;
            /* Text color change */
            transition: background-color 0.3s ease-in-out, color 0.3s ease-in-out;
            /* Smooth transition */
        }

        /* Error message text */
        .error-text {
            color: #ff3860;
            /* Error color */
            margin-top: 10px;
            /* Spacing */
            font-size: 0.9rem;
            /* Text size */
        }


        /* --- CHATBOT --- */

        /* Chatbot Button */
        #chatbotButton button {
            font-family: 'Roboto', sans-serif;
            background-color: #142941;
            /* Button background color */
            color: #E2E8F0;
            /* Button text color */
            border: 2px solid #0488D2;
            /* Border color to match the system */
            border-radius: 50px;
            /* Rounded corners */
            padding: 10px 15px;
            cursor: pointer;
            transition: background-color 0.3s, color 0.3s;
            /* Smooth transition for hover effects */
        }

        #chatbotButton button:hover {
            background-color: #0488D2;
            /* Darker color on hover for visibility */
            color: #FFFFFF;
            /* White color for text on hover */
        }

        /* Chatbot Panel */
        #chatbotPanel {
            font-family: 'Roboto', sans-serif;
            color: #000;
            /* Text color inside the chatbot panel */
            border-radius: 10px;
        }

        /* Chatbot Panel Header */
        #chatbotPanel>div:first-child {
            font-weight: bold;
            color: #E2E8F0;
            /* Header text color */
            background-color: #142941;
            /* Header background color */
            padding: 10px;
        }

        /* Close Button */
        #closeChat {
            color: #aaa;
            cursor: pointer;
        }

        #closeChat:hover {
            color: #E2E8F0;
        }

        /* Chat Content Area */
        #chatContent {
            padding: 10px;
            height: 340px;
            overflow-y: auto;
            background-color: #fff;
            /* Background color for the chat content */
            color: #000;
            /* Text color for chat content */
        }

        /* User Input Field */
        #userInput {
            width: 100%;
            box-sizing: border-box;
            border: none;
            border-top: 1px solid #ccc;
            /* Top border to separate input field from chat content */
            padding: 10px;
            background-color: #ffffff1a;
            /* Semi-transparent background */
            color: #94A3B8;
            /* Input text color */
        }

        #userInput::placeholder {
            color: #94A3B8;
            /* Placeholder text color */
        }

        /* Chat Messages */
        #chatContent div {
            margin-bottom: 5px;
            /* Space between messages */
        }

        #chatContent strong {
            color: #38BDF8;
            /* Color for the sender's name */
        }
    </style>
</head>

<body>

    <header>

        <div class="logo-container">
            <img src="logo_nobg.png" alt="LlamaCloud" class="logo">
            <!-- <span class="miniTitle">LlamaCloud</span> -->
        </div>

        <button class="login-container">
            <a href="admin" class="button-text">Admin Zone</a>
            <i class="fas fa-user-circle user-icon"></i>
        </button>


    </header>

    <main>

        <!-- Admin Password Modal -->
        <div id="adminModal" class="modal" style="display:none;">
            <div class="modal-content">
                <span class="close">&times;</span>
                <p>Please enter the admin password:</p>
                <input type="password" id="adminPassword" placeholder="Password">
                <button id="submitPassword">Enter</button>
                <p id="errorMessage" class="error-text" style="display: none;">Incorrect password.</p>
            </div>
        </div>


        <div class="container">

            <!-- WELCOME MESSAGE -->
            <div class="welcome-message">
                <h1 class="title">
                    Llama
                </h1>
                <p>The best place to search AWS Services, Documentation, Blog Posts, and More...</p>
                <p class="cta-text">Discover AWS with a simple search<br>👇🏼</p>
            </div>

            <!-- SEARCH BOX -->
            <div class="search-box">
                <button class="left-pan" aria-label="Search by voice">
                    <i class="fa fa-microphone"></i>
                    <span class="divider">|</span>
                </button>

                <input id="search" type="search" placeholder="What are you looking for in AWS?">

                <a href="search" aria-label="Search">
                    <button class="search-btn" type="submit" aria-label="Search">
                        <i class="fas fa-search"></i>
                    </button>
                </a>
            </div>

            <!-- FILTER OPTIONS -->
            <div class="filter-options">
                <button class="filter-topic">Server</button>
                <button class="filter-topic">Documents</button>
                <button class="filter-topic">Tools</button>
                <button class="filter-topic">APIs</button>
                <button class="filter-btn">Filters</button>
            </div>

            <!-- MORE FILTERS (Initially hidden)-->
            <div class="more-filters" style="display: none;">
                <div class="checkbox-container">
                    <input type="checkbox" id="option1" name="filter" value="option1">
                    <label for="option1">Option 1</label>
                </div>
                <div class="checkbox-container">
                    <input type="checkbox" id="option2" name="filter" value="option2">
                    <label for="option2">Option 2</label>
                </div>
                <div class="checkbox-container">
                    <input type="checkbox" id="option3" name="filter" value="option3">
                    <label for="option3">Option 3</label>
                </div>
            </div>


        </div>

        <!-- Chatbot Button -->
        <div id="chatbotButton" style="position: fixed; bottom: 20px; right: 20px; cursor: pointer;">
            <button>💬 Chatbot</button>
        </div>

        <!-- Chatbot Interface -->
        <div id="chatbotPanel"
            style="display: none; position: fixed; bottom: 70px; right: 20px; width: 300px; height: 400px; background-color: white; color: black; box-shadow: 0 0 10px rgba(0,0,0,0.5); border-radius: 10px; overflow: hidden;">
            <div style="padding: 10px; background-color: #142941; color: white;">
                AWS ChatBot
                <span id="closeChat" style="float: right; cursor: pointer;">&times;</span>
            </div>
            <div id="chatContent"
                style="padding: 10px; height: 340px; overflow-y: scroll; background-color: #fff; color: #000;"></div>
            <input type="text" id="userInput"
                style="width: 100%; box-sizing: border-box; border: none; border-top: 1px solid #ccc; padding: 10px;"
                placeholder="Type a message...">
        </div>

    </main>

    <footer>
        <a href="https://youtu.be/dQw4w9WgXcQ" target="_blank">made with 💙 in Braude</a>
        <a href="https://github.com/SecretPasta/Llama_Cloud.git" target="_blank">GitHub</a>
    </footer>

    <!-- SCRIPT SECTION -->

    <script>
        document.addEventListener('DOMContentLoaded', function () {
            var modal = document.getElementById("adminModal");
            var btn = document.querySelector(".login-container");
            var span = document.getElementsByClassName("close")[0];
            var passwordInput = document.getElementById("adminPassword");
            var submitBtn = document.getElementById("submitPassword");
            var errorMessage = document.getElementById("errorMessage"); // Error message element

            btn.onclick = function () {
                modal.style.display = "block";
            };

            span.onclick = function () {
                modal.style.display = "none";
            };

            window.onclick = function (event) {
                if (event.target == modal) {
                    modal.style.display = "none";
                }
            };

            submitBtn.addEventListener('click', function () {
                if (passwordInput.value === "bebra") {
                    window.location.href = "admin.html"; // Redirect to admin page
                } else {
                    errorMessage.style.display = "block"; // Show error message
                }
            });
        });


    </script>


    <!-- SEARCH QUERY -->
    <script>
        document.addEventListener('DOMContentLoaded', function () {
            var searchInput = document.getElementById('search');
            var searchBtn = document.querySelector('.search-btn');

            // Listen for the Enter key press within the search input field
            searchInput.addEventListener("keyup", function (event) {
                // Check if the key pressed is the ENTER key
                if (event.keyCode === 13) {
                    event.preventDefault(); // Prevent the default action
                    searchBtn.click(); // Trigger a click on the search button
                }
            });

            // Event listener for the search button click
            searchBtn.addEventListener('click', function () {
                var query = searchInput.value; // Get the search input value
                console.log("Search query:", query);
                // Implement the redirection to the search results page with the query
                // This is just an example path; adjust based on your actual setup

                // Store the search query in the session storage
                sessionStorage.setItem('searchQuery', query);

                window.location.href = `search_results.html`;
            });
        });
    </script>

    <!-- AUDIO INPUT -->
    <script>
        // Example function to start audio input - this will need actual implementation
        document.querySelector('.left-pan').addEventListener('click', function () {
            console.log("Audio input activated");
            // Implement audio input functionality here
        });
    </script>

    <!-- BUTTONS STAY PRESSED AFTER CLICK -->
    <script>
        // Wait for the DOM to fully load
        document.addEventListener('DOMContentLoaded', function () {
            // Get all the filter-topic buttons
            var filterButtons = document.querySelectorAll('.filter-topic');

            // Add a click event listener to each button
            filterButtons.forEach(function (button) {
                button.addEventListener('click', function () {
                    // Toggle the 'pressed' class on click
                    button.classList.toggle('pressed');
                });
            });
        });
    </script>

    <!-- OPEN FILTERS ON CLICK -->
    <script>
        document.addEventListener('DOMContentLoaded', function () {
            const filterBtn = document.querySelector('.filter-btn');
            const moreFilters = document.querySelector('.more-filters');

            filterBtn.addEventListener('click', function () {
                // Toggle visibility
                if (moreFilters.style.display === 'none') {
                    moreFilters.style.display = 'block';
                } else {
                    moreFilters.style.display = 'none';
                }
            });
        });
    </script>

    <!-- CHATBOT -->
    <script>
        document.getElementById('chatbotButton').onclick = function () {
            document.getElementById('chatbotPanel').style.display = 'block';
        };

        document.getElementById('closeChat').onclick = function () {
            document.getElementById('chatbotPanel').style.display = 'none';
        };

        document.getElementById('userInput').addEventListener('keypress', function (e) {
            if (e.key === 'Enter') {
                const userInput = this.value;
                this.value = ''; // Clear input field
                addChat("You", userInput);

                // Simulate a response
                const response = simulateResponse(userInput);
                addChat("AWS ChatBot", response);
            }
        });

        function addChat(sender, message) {
            const chatContent = document.getElementById('chatContent');
            const messageDiv = document.createElement('div');
            messageDiv.innerHTML = `<strong>${sender}:</strong> ${message}`;
            chatContent.appendChild(messageDiv);
            chatContent.scrollTop = chatContent.scrollHeight; // Scroll to the bottom
        }

        function simulateResponse(input) {
            // Simulate a basic response logic
            if (/hi|hello|hey/i.test(input)) {
                return "Hello! 😊 Welcome to AWS help chat!";
            } else if (/how are you?/i.test(input)) {
                return "I'm just a bot, so I'm always operating at peak efficiency! 😉";
            } else if (/name?/i.test(input)) {
                return "Call me AWS Bot. Your go-to assistant for all things AWS!";
            }
            // Add more response conditions here
            return "Sorry, I didn't understand that. Can you rephrase?";
        }
    </script>


</body>

</html>


"""

# Search Results HTML

In [28]:
search_results = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Llama Search Results</title>

    <script src="https://ajax.googleapis.com/ajax/libs/jquery/3.5.1/jquery.min.js"></script>

    <!-- For PNG or other formats -->
    <link rel="icon" type="image/png" href="logo_nobg.png">
    <!-- Apple Touch Icon for use with Apple devices -->
    <link rel="apple-touch-icon" href="logo_nobg.png">
    <!-- To specify a theme color for address bar in mobile browsers -->
    <meta name="theme-color" content="#10182F">

    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }

        body,
        html {
            -webkit-user-select: none;
            -moz-user-select: none;
            -ms-user-select: none;
            user-select: none;
            height: 100%;
            font-family: 'Roboto', sans-serif;
            background: linear-gradient(to bottom left, #0F172A, #1A2346, #193251, #142941, #0F172A, #0F172A, #0F172A);
            color: white;
        }

        header {
            display: flex;
            flex-direction: column;
            align-items: center;
            width: 100%;
            padding: 1rem;
        }

        .header-top {
            display: flex;
            width: 100%;
            justify-content: space-between;
            align-items: center;
        }

        .search-box {
            display: flex;
            align-items: center;
            overflow: hidden;
            border-radius: 50px;
            border: 2px solid #0488D2;
            background: #ffffff1a;
            flex-grow: 2;
            margin: 0 10px;
        }

        .search-box input[type="search"] {
            flex-grow: 1;
            padding: 10px;
            border: none;
            background-color: transparent;
            color: #94A3B8;
            font-size: 1rem;
            font-weight: bold;
        }

        .search-box .search-btn,
        .left-pan {
            background: none;
            border: none;
            padding: 10px;
            cursor: pointer;
            color: #94A3B8;
        }

        .search-box button {
            background: none;
            border: none;
            color: #94A3B8;
            cursor: pointer;
        }

        .filter-options {
            display: flex;
            justify-content: center;
            gap: 10px;
            width: 100%;
            margin-top: 10px;
        }

        .filter-topic,
        .filter-btn {
            background: #ffffff1a;
            border: 1px solid #0488D2;
            border-radius: 20px;
            padding: 5px 15px;
            cursor: pointer;
            color: #E2E8F0;
        }

        .filter-options button:hover {
            background: #0488D2;
            color: #ffffff;
        }

        header .login,
        header .user-icon {
            color: #94A3B8;
        }

        header .user-icon {
            font-size: 2rem;
        }

        .logo-container {
            display: flex;
            align-items: center;
            justify-content: flex-start;
            padding: 10px;
        }

        .logo {
            height: 80px;
            width: auto;
            margin-right: 10px;
        }

        .miniTitle {
            font-size: 1.5rem;
            font-weight: 600;
            color: #38BDF8;
        }

        .title {
            font-size: 6rem;
            font-weight: 600;
            -webkit-background-clip: text;
            -webkit-text-fill-color: transparent;
            display: inline;
            background-clip: text;
            color: transparent;
            text-shadow: 0 0 3px #007bff59, 0 0 6px #007bff56, 0 0 9px #85d6ff50;
        }

        .login-container {
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 8px;
            background-color: #ffffff0f;
            border: 1px solid #ffffff1a;
            border-radius: 50px;
            text-decoration: none;
            color: #94A3B8;
            transition: background-color 0.3s ease;
            cursor: pointer;
            font-size: 0.9rem;
        }

        .login-container i {
            margin-left: 5px;
        }

        .login-container:hover {
            background-color: #ffffff1a;
        }

        .main {
            display: flex;
            flex-direction: column;
            justify-content: space-between;
            /* Distributes space between table and pagination */
            align-items: center;
            /* Center items horizontally */
            width: 100%;
            height: 100vh;
            /* Take up the full viewport height */
            padding-top: 1rem;
            padding-bottom: 1rem;
            /* Add padding at the bottom */
            box-sizing: border-box;
            /* Include padding in the height calculation */
        }

        .search-container {
            display: flex;
            flex-direction: column;
            flex-grow: 1;
            /* Allow the container to grow and fill available space */
            width: 90vw;
            /* Adjust based on your preference */
            margin: 20px auto;
            padding: 20px;
            background-color: #ffffff1a;
            border-radius: 8px;
            box-shadow: 0 4px 8px rgba(0, 0, 0, 0.1);
            overflow: hidden;
            /* Contain the table's overflow */
        }

        .table-container {
            flex-grow: 1;
            /* Allow the table container to grow */
            width: 100%;
            /* Full width to match the .search-container */
            height: 500px;
            /* Fixed height; adjust as needed */
            overflow: auto;
            /* Enables scrolling for overflow content */
            margin-bottom: 20px;
            /* Space above pagination */
        }


        /* Table General Styles */
        table {
            width: 100%;
            /* Full width of its container */
            border-collapse: collapse;
            /* Clean table borders */
            table-layout: fixed;
            /* Fixes column widths */
        }

        th,
        td {
            padding: 12px;
            /* Adequate padding for cell content */
            text-align: left;
            /* Align text to the left for readability */
            border-bottom: 1px solid #ddd;
            /* Light border for row separation */
        }

        /* Title and Description Styles within Table Cells */
        /* Assuming your titles are links (<a>), and descriptions are in <div> or <p> */
        td a,
        td div,
        td p {
            display: block;
            /* Ensure block-level display for proper spacing */
        }

        td a {
            color: #0488D2;
            /* Title (link) color to match your scheme */
            font-weight: bold;
            /* Make titles stand out */
            text-decoration: none;
            /* No underline for a clean look */
            margin-bottom: 5px;
            /* Space between title and description */
        }

        td a:hover {
            text-decoration: underline;
            /* Underline on hover for links */
        }

        td div,
        td p {
            color: #cbcbcb;
            /* Subdued color for descriptions */
            font-size: 0.95rem;
            /* Slightly smaller font for descriptions */
        }

        /* Rank Styles */
        td:last-child {
            width: 10%;
            /* Allocate width to the rank column */
            text-align: right;
            /* Align rank to the right */
            font-weight: bold;
            /* Make the rank stand out */
            color: #ffffff;
            /* Darker text for better visibility */
        }

        th:nth-child(2) {
            text-align: right;
        }


        /* Style the scrollbar itself (background) */
        .table-container::-webkit-scrollbar {
            width: 8px;
            /* Adjust the width of the vertical scrollbar */
            height: 8px;
            /* Adjust the height of the horizontal scrollbar */
            background-color: #f0f2f5;
            /* Scrollbar background */
        }

        /* Style the scrollbar thumb (the part that you drag) */
        .table-container::-webkit-scrollbar-thumb {
            background-color: #0488D2;
            /* Thumb color */
        }

        /* Style the scrollbar thumb on hover */
        .table-container::-webkit-scrollbar-thumb:hover {
            background-color: #036C9E;
            /* Darker shade for hover effect */
        }


        #pagination {
            display: flex;
            justify-content: center;
            align-items: center;
            gap: 10px;
            flex-wrap: nowrap;
            white-space: nowrap;
            overflow-x: auto;
            padding: 10px 0;
        }

        #pagination button,
        #pagination span {
            color: #0488D2;
            /* Set text color for both buttons and the page indicator */
            padding: 10px 15px;
            /* Apply padding to both buttons and span for consistent size */
            min-width: 80px;
            /* Ensure buttons have at least a minimum width */
            text-align: center;
            /* Center text within buttons and span */
            background-color: transparent;
            border-radius: 5px;
            cursor: pointer;
            transition: background-color 0.3s, color 0.3s;
        }

        #pagination button {
            border: 1px solid #0488D2;
            /* Only buttons have a border */
            color: #ffffff;
        }

        #pagination button:hover {
            background-color: #007bff;
            color: #ffffff;
            /* Text color change on hover for buttons */
        }

        #pagination span {
            cursor: default;
            /* The page indicator shouldn't change cursor like buttons */
            background-color: transparent;
            /* Ensure the span doesn't have a background */
            border: none;
            /* Remove border for the span to avoid visual inconsistency */
            display: inline-flex;
            /* Align text vertically */
            align-items: center;
            /* Center text vertically */
            justify-content: center;
            /* Center text horizontally */
        }



        footer {
            position: absolute;
            bottom: 0;
            width: 100%;
            padding: 1rem;
            text-align: center;
            background: transparent;
        }

        footer a {
            color: #E2E8F0;
            text-decoration: none;
            font-size: 1rem;
            margin: 0 1rem;
            transition: color 0.3s ease;
        }

        footer a:hover {
            color: #0E99DA;
        }
    </style>
</head>

<body>
    <header>
        <div class="header-top">
            <div class="logo-container">
                <img src="logo_nobg.png" alt="LlamaCloud" class="logo">
            </div>
            <div class="search-box">
                <button class="left-pan" aria-label="Search by voice">
                    <i class="fa fa-microphone"></i>
                    <span class="divider">|</span>
                </button>
                <input id="search" type="search" placeholder="What are you looking for in AWS?">
                <button class="search-btn" type="submit" aria-label="Search">
                    <i class="fas fa-search"></i>
                </button>
            </div>
            <a href="index.html" class="login-container">Log Out
                <i class="fas fa-sign-out-alt"></i>
            </a>
        </div>
        <div class="filter-options">
            <button class="filter-topic">Server</button>
            <button class="filter-topic">Documents</button>
            <button class="filter-topic">Tools</button>
            <button class="filter-topic">APIs</button>
            <button class="filter-btn">Filters</button>
        </div>
    </header>

    <main>
        <div class="search-container">
            <h2 class="results-title"></h2>
            <div class="table-container">
                <table id="resultsTable">
                    <thead>
                        <tr>
                            <th>Title and Description</th>
                            <th>Rank</th>
                        </tr>
                    </thead>
                    <tbody>
                        <!-- Results will be inserted here via JavaScript -->
                    </tbody>
                </table>
            </div>

        </div>

        <div id="pagination">
            <button id="prevPage">Previous</button>
            <span id="pageIndicator">Page 1</span>
            <button id="nextPage">Next</button>
        </div>
    </main>

    <footer>
        <a href="https://youtu.be/dQw4w9WgXcQ" target="_blank">made with 💙 in Braude</a>
        <a href="https://github.com/SecretPasta/Llama_Cloud.git" target="_blank">GitHub</a>
    </footer>

    <!-- SCRIPT SECTION -->


    <!-- DECODE QUERY -->
    <script>

        $(document).ready(function () {
            var searchQuery = sessionStorage.getItem('searchQuery');
            console.log(searchQuery); // Confirming the search query is retrieved correctly
            // Update the text of the results-title element with the searchQuery
            $(".results-title").text("Search Results for " + searchQuery);

            fetch('index.json')
                .then(response => response.json())
                .then(jsonData => {
                    let termData = jsonData.index["-NtaO-5dL_XJw3XQ3bcE"].term[searchQuery] || [];
                    // Ensure sorting is in descending order based on rank
                    termData.sort((a, b) => b.rank - a.rank);

                    // Limit to top 100 results
                    termData = termData.slice(0, 100);

                    let currentPage = 1;
                    const resultsPerPage = 10;
                    const totalPages = Math.ceil(termData.length / resultsPerPage);

                    function displayPage(page) {
                        const start = (page - 1) * resultsPerPage;
                        const end = start + resultsPerPage;
                        const pageItems = termData.slice(start, end);

                        const $tbody = $('#resultsTable tbody');
                        $tbody.empty(); // Clear previous results

                        $.each(pageItems, function (index, item) {
                            $tbody.append(`
        <tr>
            <td style="width: 90%;"><!-- Width allocation for responsiveness -->
                <div style="margin-bottom: 5px;"><a href="${item.DocIDs}" target="_blank">${item.title}</a></div>
                <div>${item.content}</div>
            </td>
            <td style="text-align: right; vertical-align: top;">${item.rank}</td>
        </tr>
    `);
                        });

                        $('#pageIndicator').text(`Page ${page} of ${totalPages}`);
                    }

                    displayPage(currentPage);

                    $('#prevPage').click(function () {
                        if (currentPage > 1) {
                            currentPage--;
                            displayPage(currentPage);
                        }
                    });

                    $('#nextPage').click(function () {
                        if (currentPage < totalPages) {
                            currentPage++;
                            displayPage(currentPage);
                        }
                    });

                })
                .catch(error => {
                    console.error('Error loading index.json:', error);
                });
        });



    </script>

</body>

</html>
"""

# Admin Section HTML

In [29]:
admin_page = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Llama - Admin Dashboard</title>

    <!-- Icons -->
    <link rel="icon" type="image/png" href="logo_nobg.png">
    <link rel="apple-touch-icon" href="logo_nobg.png">
    <meta name="theme-color" content="#10182F">

    <!-- Fonts and Icons -->
    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">

    <!-- Visibility Change Script -->
    <script>
        function handleVisibilityChange() {
            if (document.hidden) {
                document.title = "Llama 😢";
            } else {
                document.title = "Llama";
            }
        }
        document.addEventListener("visibilitychange", handleVisibilityChange, false);
    </script>

    <style>
        body,
        html {
            margin: 0;
            padding: 0;
            overflow-x: hidden;
            font-family: 'Roboto', sans-serif;
            background: linear-gradient(to bottom left, #0F172A, #1A2346, #193251, #142941, #0F172A);
            color: white;
            -webkit-user-select: none;
            -moz-user-select: none;
            -ms-user-select: none;
            user-select: none;
            background-attachment: fixed;

            height: 100%;
            min-height: 100vh;
        }

        /* --- HEADER --- */
        header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            width: 100%;
            padding: 0.5rem;
        }

        .logo-container,
        .login-container {
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 8px;
            gap: 6px;
        }

        .logo {
            height: 70px;
            width: auto;
        }

        .section-title {
            color: #007bff;
        }

        .miniTitle {
            width: 100%;
            text-align: center;
            font-size: 1.5rem;
            font-weight: 600;
            color: #38BDF8;
            margin: 0;
            padding: 0;
        }

        .login-container {
            text-decoration: none;
            display: flex;
            align-items: center;
            justify-content: center;
            color: #94A3B8;
            background-color: #ffffff0f;
            border: 1px solid #ffffff1a;
            border-radius: 10px;
            padding: 10px;
            margin: 20px;
            backdrop-filter: blur(10px);
            transition: background-color 0.3s ease;
            cursor: pointer;

            width: auto;
            min-width: 120px;
        }

        .login-container:hover {
            background-color: #ffffff1a;
        }

        .main {
            display: flex;
            flex-direction: column;
            align-items: center;
            padding: 20px;
            gap: 20px;
        }

        .admin-container,
        .dashboard-section {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            text-align: center;
            margin-bottom: 40px;
        }

        .dashboard-section {
            background: #ffffff1a;
            /* Light background for sections */
            border-radius: 10px;
            padding: 20px;
            margin: 10px 0;
            width: 80%;
            /* Adjusted width for better layout control */
            max-width: 960px;
            /* Max width for larger screens */
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            /* Subtle shadow for depth */
        }

        .action-btn {
            background-color: #007bff;
            color: #FFFFFF;
            border: none;
            border-radius: 5px;
            padding: 10px 15px;
            cursor: pointer;
            transition: background 0.3s ease;
            align-self: center;
            /* Center button in the section */
            text-decoration: none;
        }

        .action-btn:hover {
            background-color: #0056b3;
        }

        .code-window {
            background-color: #0F172A;
            color: #dcdcdc;
            padding: 15px;
            border-radius: 8px;
            overflow: auto;
            font-size: 0.9rem;
            font-family: 'Courier New', Courier, monospace;
            white-space: pre-wrap;
            text-align: left;
        }

        .scan-info {
            display: flex;
            justify-content: space-between;
            align-items: center;
            width: 90%;
            max-width: 800px;
            margin-top: 20px;
        }

        .scan-status {
            color: #4CAF50;
            font-weight: bold;
        }

        /* --- FOOTER --- */

        footer {
            bottom: 0;
            width: 100%;
            padding: 1rem;
            text-align: center;
            background: transparent;
        }

        footer a {
            color: #E2E8F0;
            /* Light grey color for footer links */
            text-decoration: none;
            font-size: 1rem;
            margin: 0 1rem;
            transition: color 0.3s ease;
        }

        footer a:hover {
            color: #0E99DA;
        }


        /* --- FOR DIFFERENT SIZES OF SCREEN --- */

        @media (max-width: 600px) {
            .scan-info {
                flex-direction: column;
                align-items: flex-start;
            }

            .scan-info span,
            .scan-info button {
                margin-bottom: 10px;
            }
        }

        @media (max-width: 768px) {
            .dashboard-section {
                width: 95%;
                /* More width for smaller screens */
            }

            .logo {
                height: 40px;
                /* Smaller logo for smaller screens */
            }

            .section-title,
            .code-window,
            .action-btn {
                font-size: 0.8rem;
                /* Smaller text for better readability */
            }
        }
    </style>
</head>

<body>

    <header>
        <div class="logo-container">
            <img src="logo_nobg.png" alt="LlamaCloud" class="logo">
        </div>

        <span class="miniTitle">Hello, Admin 👋🏼</span>

        <a href="" class="login-container">Log Out
            <i class="fas fa-sign-out-alt"></i>
        </a>
    </header>

    <main>
        <div class="admin-container">
            <!-- Dashboard Sections -->
            <section class="dashboard-section">
                <h2 class="section-title">Check Statistic</h2>
                <a onclick = "openStats()" class="action-btn">Check Statistic</a>
            </section>

            <section class="dashboard-section">
                <h2 class="section-title">Index of AWS</h2>
                <pre class="code-window">
{
    "AWSSearch": {
        "service": "EC2",
        "documentation": "Instance Types",
        "blogPosts": [
            "Scaling Your EC2 Infrastructure",
            "Cost Optimization with EC2"
        ],
        "moreInfo": "Visit AWS official documentation for detailed information."
    }
}
            </pre>
            </section>

            <section class="dashboard-section">
                <h2 class="section-title">Last scan of AWS</h2>
                <div class="scan-info">
                    <span id="lastScanDate">Last Scan of AWS on 20/02/2024</span>
                    <span id="scanStatus" class="scan-status success">Success</span> <!-- Dynamic status indicator -->
                    <button onclick = "scrapeAndSave()" class="action-btn">Scan AWS</button>
                </div>
            </section>
        </div>
    </main>

    <footer>
        <a href="https://youtu.be/dQw4w9WgXcQ" target="_blank">made with 💙 in Braude</a>
        <a href="https://github.com/SecretPasta/Llama_Cloud.git" target="_blank">GitHub</a>
    </footer>

<script>
      function openMainPage() {
    // This will call the Python function 'display_homepage'
    google.colab.kernel.invokeFunction('notebook.display_homepage', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

<script>
      function openStats() {
    // This will call the Python function 'display_stats'
    google.colab.kernel.invokeFunction('notebook.display_stats', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

<script>
      function scrapeAndSave() {
    // This will call the Python function 'scrape_and_save'
    google.colab.kernel.invokeFunction('notebook.scrape_and_save', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

</body>

</html>
"""

# Statistics Page HTML

In [11]:
stats_page = """
<!DOCTYPE html>
<html lang="en">

<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Llama - Statistics</title>

    <!-- For PNG or other formats -->
    <link rel="icon" type="image/png" href="logo_nobg.png">

    <link href="https://fonts.googleapis.com/css2?family=Roboto:wght@400;700&display=swap" rel="stylesheet">
    <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/font-awesome/5.15.3/css/all.min.css">
    <style>
        * {
            box-sizing: border-box;
            margin: 0;
            padding: 0;
        }

        body,
        html {

            -webkit-user-select: none;
            /* Safari */
            -moz-user-select: none;
            /* Firefox */
            -ms-user-select: none;
            /* Internet Explorer/Edge */
            user-select: none;
            /* Non-prefixed version, currently supported by Chrome, Opera and Edge */

            height: 100%;
            font-family: 'Roboto', sans-serif;
            background: linear-gradient(to bottom left, #0F172A, #1A2346, #193251, #142941, #0F172A, #0F172A, #0F172A);
            /* Dark background */
            background-attachment: fixed;
            /* Keep the gradient fixed during scroll */
            color: white;
        }

        /* --- HEADER --- */
        header {
            display: flex;
            justify-content: space-between;
            align-items: center;
            width: 100%;
            padding: 0.5rem;
        }

        .logo-container,
        .login-container {
            display: flex;
            align-items: center;
            justify-content: center;
            padding: 8px;
            gap: 6px;
        }

        .logo {
            height: 70px;
            width: auto;
        }

        .section-title {
            color: #007bff;
        }

        .miniTitle {
            width: 100%;
            text-align: center;
            font-size: 1.5rem;
            font-weight: 600;
            color: #38BDF8;
            margin: 0;
            padding: 0;
        }

        .login-container {
            text-decoration: none;
            display: flex;
            align-items: center;
            justify-content: center;
            color: #94A3B8;
            background-color: #ffffff0f;
            border: 1px solid #ffffff1a;
            border-radius: 10px;
            padding: 10px;
            margin: 20px;
            backdrop-filter: blur(10px);
            transition: background-color 0.3s ease;
            cursor: pointer;

            width: auto;
            min-width: 120px;
        }

        .login-container:hover {
            background-color: #ffffff1a;
        }

        /* --- MAIN --- */

        .main {
            display: flex;
            justify-content: center;
            align-items: center;
            min-height: 100vh;
            /* Use at least 100% of the viewport height */
        }

        .stat-container {
            display: flex;
            flex-direction: column;
            align-items: center;
            justify-content: center;
            text-align: center;
            margin-bottom: 40px;
        }

        .dashboard-section {
            background: #ffffff1a;
            /* Light background for sections */
            border-radius: 10px;
            padding: 20px;
            margin: 20px 0;
            width: 80%;
            /* Adjusted width for better layout control */
            max-width: 960px;
            /* Max width for larger screens */
            box-shadow: 0 2px 4px rgba(0, 0, 0, 0.1);
            /* Subtle shadow for depth */
        }


        .section-title {
            text-align: center;
            margin-bottom: 15px;
        }

        /* Line Chart Styling */

        /* --- BAR CHART --- */
        .simple-bar-chart {
            --line-count: 10;
            --line-color: currentcolor;
            --line-opacity: 0.25;
            --item-gap: 2%;
            --item-default-color: #060606;

            height: 10rem;
            display: grid;
            grid-auto-flow: column;
            gap: var(--item-gap);
            align-items: end;
            padding-inline: var(--item-gap);
            --padding-block: 1.5rem;
            /*space for labels*/
            padding-block: var(--padding-block);
            position: relative;
            isolation: isolate;
        }

        .simple-bar-chart::after {
            content: "";
            position: absolute;
            inset: var(--padding-block) 0;
            z-index: -1;
            --line-width: 1px;
            --line-spacing: calc(100% / var(--line-count));
            background-image: repeating-linear-gradient(to top, transparent 0 calc(var(--line-spacing) - var(--line-width)), var(--line-color) 0 var(--line-spacing));
            box-shadow: 0 var(--line-width) 0 var(--line-color);
            opacity: var(--line-opacity);
        }

        .simple-bar-chart>.item {
            height: calc(1% * var(--val));
            background-color: var(--clr, var(--item-default-color));
            position: relative;
            animation: item-height 1s ease forwards
        }

        @keyframes item-height {
            from {
                height: 0
            }
        }

        .simple-bar-chart>.item>* {
            position: absolute;
            text-align: center
        }

        .simple-bar-chart>.item>.label {
            inset: 100% 0 auto 0
        }

        .simple-bar-chart>.item>.value {
            inset: auto 0 100% 0
        }

        /* >--- BAR CHART ---< */

        /* --- PIE CHART --- */

        .pie-chart-container {
            position: relative;
            display: flex;
            justify-content: center;
            align-items: center;
        }

        .pie-chart {
            width: 300px;
            height: 300px;
            border-radius: 50%;
            position: relative;
            background-color: #eee;
            /* Fallback color */
            background-image:
                conic-gradient(from 0deg at 50% 50%,
                    #FFD700 0deg 90deg,
                    /* Chrome */
                    #FF8C00 90deg 162deg,
                    /* Firefox */
                    #FF6347 162deg 207deg,
                    /* Edge */
                    #4682B4 207deg 360deg
                    /* Safari */
                );
        }

        .segment {
            position: absolute;
            width: 100%;
            height: 100%;
            clip-path: circle(50% at 50% 50%);
            display: flex;
            justify-content: center;
            align-items: center;
            text-align: justify;
            font-size: 1;
            /* Hide text */
        }

        .segment::before {
            content: attr(style);
            font-size: 14px;
            position: absolute;
            white-space: nowrap;
            color: transparent;
            /* Hide actual content */
        }

        .labels {
            position: absolute;
            width: 100%;
            height: 100%;
            top: 0;
            left: -30px;
            display: flex;
            justify-content: center;
            align-items: center;
            color: black;
        }

        .label {
            position: absolute;
            /* Adjust the translate value to move labels outward */
            transform: translate(100%, -50%) rotate(var(--angle)) translateX(50%);
            transform-origin: 0% 50%;
            white-space: nowrap;
        }


        /* >--- PIE CHART ---< */



        /* --- FOOTER --- */

        footer {
            bottom: 0;
            width: 100%;
            padding: 1rem;
            text-align: center;
            background: transparent;
        }

        footer a {
            color: #E2E8F0;
            /* Light grey color for footer links */
            text-decoration: none;
            font-size: 1rem;
            margin: 0 1rem;
            transition: color 0.3s ease;
        }

        footer a:hover {
            color: #0E99DA;
        }
    </style>
</head>

<body>

    <header>
        <div class="logo-container">
            <img src="logo_nobg.png" alt="LlamaCloud" class="logo">
        </div>

        <span class="miniTitle">Statistics 📊</span>

        <a onclick= "openAdmin()" class="login-container">Back
            <i class="fas fa-arrow-left"></i>
        </a>
    </header>

    <main>
        <div class="stat-container">

            <!-- Monthly Usage Bar Chart -->
            <div class="dashboard-section">
                <h2 class="section-title">Monthly Usage</h2>
                <div class="simple-bar-chart">
                    <!-- Each "item" represents a month with a unique color and value -->

                    <div class="item" style="--clr: #5EB344; --val: 21"> <!-- Example for January -->
                        <div class="label">Jan</div>
                        <div class="value">21%</div>
                    </div>

                    <div class="item" style="--clr: #FCB72A; --val: 43"> <!-- Example for February -->
                        <div class="label">Feb</div>
                        <div class="value">43%</div>
                    </div>

                    <div class="item" style="--clr: #F8821A; --val: 65"> <!-- Example for March -->
                        <div class="label">Mar</div>
                        <div class="value">65%</div>
                    </div>

                    <div class="item" style="--clr: #E0393E; --val: 53"> <!-- Example for April -->
                        <div class="label">Apr</div>
                        <div class="value">53%</div>
                    </div>
                </div>
            </div>

            <!-- Browser Usage Pie Chart -->
            <div class="dashboard-section">
                <h2 class="section-title">Browser Usage</h2>
                <div class="pie-chart-container">
                    <div class="pie-chart">
                        <div class="segment chrome" data-name="Chrome"></div>
                        <div class="segment firefox" data-name="Firefox"></div>
                        <div class="segment edge" data-name="Edge"></div>
                        <div class="segment safari" data-name="Safari"></div>
                    </div>
                    <div class="labels">
                        <div class="label" style="--angle: 0;">Chrome 25%</div>
                        <div class="label" style="--angle: 90deg;">Firefox 20%</div>
                        <div class="label" style="--angle: 180deg;">Edge 15%</div>
                        <div class="label" style="--angle: 270deg;">Safari 40%</div>
                    </div>
                </div>

            </div>

        </div>
    </main>




    <footer>
        <a href="https://youtu.be/dQw4w9WgXcQ" target="_blank">made with 💙 in Braude</a>
        <a href="https://github.com/SecretPasta/Llama_Cloud.git" target="_blank">GitHub</a>
    </footer>

    <script>
      function openAdmin() {
    // This will call the Python function 'display_admin'
    google.colab.kernel.invokeFunction('notebook.display_admin', [], {})
        .then(function(response) {
            // Handle response here if needed
        })
        .catch(function(error) {
            console.error(error);
        });
}
</script>

</body>

</html>
"""

# Python Code

Methods for swapping screens, invoked from JS code in HTML

In [12]:
def display_homepage():
  clear_output(wait=True)
  display(HTML(landing_page_html))

In [13]:
def display_admin():
  clear_output(wait=True)
  display(HTML(admin_page))

In [14]:
def display_stats():
  clear_output(wait=True)
  display(HTML(stats_page))

In [15]:
def display_results():
    clear_output(wait=True)
    display(HTML(search_results))

Code to have functions called from JavaScript code inside HTML files

In [16]:
output.register_callback('notebook.display_results', display_results)
output.register_callback('notebook.display_homepage', display_homepage)
output.register_callback('notebook.display_admin', display_admin)
output.register_callback('notebook.display_stats', display_stats)
output.register_callback('notebook.scrape_and_save', scrape_and_save)

# Main

In [18]:
from google.colab import output
output.serve_kernel_port_as_window(8084)

<IPython.core.display.Javascript object>

In [33]:
from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return landing_page_html

@app.route('/search')
def search():
    return search_results

@app.route('/admin')
def admin():
    return admin_page

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8084)

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:8084
 * Running on http://172.28.0.12:8084
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [22/Mar/2024 18:50:07] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2024 18:50:07] "GET /logo_nobg.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2024 18:50:08] "GET /logo_nobg.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2024 18:50:11] "GET /search_results.html HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2024 18:50:11] "GET /search HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2024 18:50:11] "GET /logo_nobg.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2024 18:50:11] "GET /index.json HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2024 18:50:11] "GET /logo_nobg.png HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [22/Mar/2024 